In [6]:
import shutil
import os 
import json
import re

# from general_json2yolo import *

# convert_coco_json("cocodoomData/run1/log.txt")
# convert_ath_json("cocodoomData\map-train.json")

In [ ]:
# rn = "3"
# for dir in next(os.walk('cocodoomData/run'+rn))[1]:
#     print(dir)
#     os.mkdir("yolo/images/run"+rn+"/"+dir)
#     os.mkdir("yolo/labels/run"+rn+"/"+dir)

In [ ]:
f = json.load(open("cocodoomData/run-train.json","r"))
idImagesLink = {}
for images in f["images"]:
    idImagesLink[images["id"]] = {"file_name":images["file_name"], "bbox":[], "category_id":[]}

for annotation in f["annotations"]:
    idImagesLink[annotation["image_id"]]["bbox"].append(annotation["bbox"])
    idImagesLink[annotation["image_id"]]["category_id"].append(annotation["category_id"])
print(idImagesLink.keys())
print(idImagesLink[1030004982])



In [9]:


# print(f.keys())
# print(type(f['images']))
# print(f["annotations"][0])

# print(len(f["annotations"]))
# print(len(f["images"]))



# print(idImagesLink[list(idImagesLink.keys())[0]])

src = "cocodoomData/"
dest = "yolo/"
width = 320
height = 200
names = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 106, 108, 109, 110, 117, 118, 119, 121, 123, 124, 125, 126, 127, 134]

for fname in ["run-train.json","run-val.json", "run-test.json"]:
    f = json.load(open("cocodoomData/"+fname,"r"))
    idImagesLink = {}
    for images in f["images"]:
        idImagesLink[images["id"]] = {"file_name":images["file_name"], "bbox":[], "category_id":[]}

    for annotation in f["annotations"]:
        idImagesLink[annotation["image_id"]]["bbox"].append(annotation["bbox"])
        idImagesLink[annotation["image_id"]]["category_id"].append(annotation["category_id"])

    for elem in idImagesLink:
        dest = "yolo/"+idImagesLink[elem]["file_name"].replace("/rgb","images")
        dest = re.sub(r'\bmap\d{1,2}', '', dest)
        # shutil.copy(src+idImagesLink[elem]["file_name"], dest)
        with open(dest.replace("images","labels").replace(".png",".txt"),"w") as f:
            # if len(idImagesLink[elem]["bbox"]) == 0:
            #     f.write(str(0)+"\n")
            for i, bbox in enumerate(idImagesLink[elem]["bbox"]):
                xmin = bbox[0]
                ymin = bbox[1]
                xmax = bbox[0]+bbox[2]
                ymax = bbox[1]+bbox[3]
                center_x = min(((xmin+xmax)//2)/width,1.0)
                center_y = min(((ymin+ymax)//2)/height,1.0)
                widthbb = min((xmax-xmin)/width,1)
                heightbb = min((ymax-ymin)/height,1)
                f.write(str(names.index(idImagesLink[elem]["category_id"][i]))+" "+str(center_x)+" "+str(center_y)+" "+str(widthbb)+" "+str(heightbb)+"\n")

                # f.write(str(idImagesLink[elem]["category_id"][i])+" "+str(center_x)+" "+str(center_y)+" "+str(widthbb)+" "+str(heightbb)+"\n")

In [2]:
import yaml

data = dict(
path=  'C:/Users/engel/Documents/cocoDoom/yolo',
train= "C:/Users/engel/Documents/cocoDoom/yolo/run1/images",
test= "C:/Users/engel/Documents/cocoDoom/yolo/run3/images",
val= "C:/Users/engel/Documents/cocoDoom/yolo/run2/images",
nc=94,
names = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 106, 108, 109, 110, 117, 118, 119, 121, 123, 124, 125, 126, 127, 134]
)

with open('data.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [11]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
model.train(data='data.yml', epochs=3, imgsz=640)

Ultralytics YOLOv8.0.98  Python-3.10.0 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.0

KeyboardInterrupt: 

In [ ]:
f = json.load(open("cocodoomData/run-train.json","r"))
print(f["annotations"][1])